In [72]:
# https://ipython.org/ipython-doc/3/config/extensions/``autoreload.html
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
from termcolor import cprint
from load_path import *
from run import get_itj_pddl_problem_from_json

In [74]:
debug_problem_name = "nine_pieces_process_symbolic.json"
input_problem = get_itj_pddl_problem_from_json(debug_problem_name, use_partial_order=True, debug=True, 
                                                    reset_to_home=0,
                                                    use_fluents=1)

Symbolic process json parsed from C:\Users\harry\Documents\code_ws\pb_ws\debug_pyplanners\itj\nine_pieces_process_symbolic.json
Using beam sequence ordering: ['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']


In [4]:
from pddlstream.algorithms.algorithm import parse_problem
from pddlstream.algorithms.constraints import PlanConstraints
from pddlstream.utils import INF, Verbose, str_from_object, elapsed_time

constraints = PlanConstraints()
unit_costs = True
max_iterations=INF
max_time=INF
max_memory=INF

In [5]:
from pddlstream.algorithms.incremental import solve_incremental
from pddlstream.language.constants import print_plan, is_plan

solution = solve_incremental(input_problem, constraints=constraints,
                  unit_costs=1, success_cost=INF,
                  max_iterations=INF, max_time=INF, max_memory=INF,
                  initial_complexity=0, complexity_step=1, max_complexity=INF,
                  verbose=False)

plan, cost, evaluations = solution
plan_success = is_plan(plan)

print('-'*10)
print_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

Iteration: 1 | Complexity: 0 | Calls: 0 | Evaluations: 161 | Solved: False | Cost: inf | Search Time: 0.004 | Sample Time: 0.000 | Time: 0.004
Initial: State{Atom notoolatjoint(v1, v3), Atom notoolatjoint(v1, v4), Atom notoolatjoint(v1, v6), Atom notoolatjoint(v2, v3), Atom notoolatjoint(v2, v4), Atom notoolatjoint(v2, v8), Atom notoolatjoint(v3, v1), Atom notoolatjoint(v3, v2), Atom notoolatjoint(v4, v1), Atom notoolatjoint(v4, v2), Atom notoolatjoint(v5, v6), Atom notoolatjoint(v5, v9), Atom notoolatjoint(v6, v1), Atom notoolatjoint(v6, v5), Atom notoolatjoint(v7, v8), Atom notoolatjoint(v7, v9), Atom notoolatjoint(v8, v2), Atom notoolatjoint(v8, v7), Atom notoolatjoint(v9, v5), Atom notoolatjoint(v9, v7)}
Goal: PartialState{Atom assembled(v1), Atom assembled(v2), Atom assembled(v3), Atom assembled(v4), Atom assembled(v5), Atom assembled(v6), Atom assembled(v7), Atom assembled(v8), Atom assembled(v9)}
Actions: 0 | Axioms: 73
Search: eager | Evaluator: greedy | Heuristic: ff | Success

KeyboardInterrupt: Interrupted by user

## Debug types

In [10]:
from pddlstream.algorithms.downward import is_valid_plan, apply_action, task_from_domain_problem, get_problem, \
    get_action_instances
from pddlstream.algorithms.algorithm import parse_problem
from pddlstream.algorithms.constraints import PlanConstraints
from pddlstream.language.attachments import has_attachments, compile_fluents_as_attachments, solve_pyplanners

evaluations, goal_expression, domain, externals = parse_problem(
    input_problem, constraints=PlanConstraints(), unit_costs=True)

In [11]:
from pddlstream.algorithms.refinement import optimistic_process_streams
from pddlstream.algorithms.scheduling.recover_streams import evaluations_from_stream_plan
from pddlstream.algorithms.instantiate_task import instantiate_task

results, exhausted = optimistic_process_streams(evaluations, externals, complexity_limit=INF, max_effort=None)
exh_evaluations = evaluations_from_stream_plan(evaluations, results, max_effort=None)

task = task_from_domain_problem(domain, get_problem(exh_evaluations, goal_expression, domain, unit_costs))

In [12]:
instantiated = instantiate_task(task)


Generating Datalog program... [0.000s CPU, 0.012s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.016s CPU, 0.007s wall-clock]
Preparing model... [0.000s CPU, 0.002s wall-clock]
Generated 43 rules.
Computing model... [0.000s CPU, 0.010s wall-clock]
889 relevant atoms
34 auxiliary atoms
923 final queue length
2192 total queue pushes
Completing instantiation... [0.016s CPU, 0.012s wall-clock]
Infeasible: False
Instantiation time: 0.047s
Instantiated frequencies:
Atoms: {assembled: 9, atrack: 25, attached: 25, canfreemove: 1, eithergroundedalltoolatjoints: 25, existnotoolatjoints: 9, new-axiom@0: 8, notoolatjoint: 20, robotgripperempty: 1, robottoolchangerempty: 1, toolatjoint: 40, toolnotoccupiedonjoint: 4}
Actions: {pick_clamp_from_joint: 40, pick_element_from_rack: 9, pick_tool_from_rack: 7, place_clamp_at_joint: 80, place_element_on_structure: 27, place_tool_at_rack: 11}
Axioms: {eithergroundedalltoolatjoints: 13, existnotoolatjoints: 20, new-axiom@0: 8}


In [9]:
atoms = list(instantiated.atoms)
atoms[0].predicate

'notoolatjoint'

In [13]:
from pddlstream.language.conversion import is_atom, is_negated_atom, objects_from_evaluations, pddl_from_object, \
    pddl_ist_from_expression, obj_from_pddl

for a in instantiated.atoms:
    print([obj_from_pddl(n) for n in a.args])

ImportError: cannot import name 'pddl_ist_from_expression' from 'pddlstream.language.conversion' (C:\Users\harry\Documents\code_ws\pb_ws\coop_assembly\external\pddlstream\pddlstream\language\conversion.py)

In [50]:
from pddlstream.language.conversion import is_atom, is_negated_atom, objects_from_evaluations, pddl_from_object, \
    pddl_list_from_expression, obj_from_pddl

# pddl_from_object()
obj_from_pddl('v1')

b0

In [20]:
from pddlstream.algorithms.downward import parse_action

parse_action(instantiated.actions[-1].name).args[0]

'v13'

In [31]:
from pddlstream.language.object import Object, OptimisticObject

parse_action(instantiated.actions[0].name).args[0]
# obj_from_pddl(parse_action(instantiated.actions[-1].name).args[0])

'#o27'

In [35]:
parse_action(instantiated.actions[0].name)
# instantiated.actions[0]

Action(name='move', args=('#o27', '#o27', '#o36'))

In [63]:
instantiated.task.init

[<Atom robotconf(v1)>,
 <Atom robotatconf(v1)>,
 <Atom canfreemove()>,
 <Atom robottoolchangerempty()>,
 <Atom element(v2)>,
 <Atom atrack(v2)>,
 <Atom iselement(v2)>,
 <Atom grounded(v2)>,
 <Atom element(v3)>,
 <Atom atrack(v3)>,
 <Atom iselement(v3)>,
 <Atom grounded(v3)>,
 <Atom element(v4)>,
 <Atom atrack(v4)>,
 <Atom iselement(v4)>,
 <Atom element(v5)>,
 <Atom atrack(v5)>,
 <Atom iselement(v5)>,
 <Atom element(v6)>,
 <Atom atrack(v6)>,
 <Atom iselement(v6)>,
 <Atom grounded(v6)>,
 <Atom element(v7)>,
 <Atom atrack(v7)>,
 <Atom iselement(v7)>,
 <Atom element(v8)>,
 <Atom atrack(v8)>,
 <Atom iselement(v8)>,
 <Atom grounded(v8)>,
 <Atom element(v9)>,
 <Atom atrack(v9)>,
 <Atom iselement(v9)>,
 <Atom element(v10)>,
 <Atom atrack(v10)>,
 <Atom iselement(v10)>,
 <Atom joint(v2, v4)>,
 <Atom joint(v4, v2)>,
 <Atom notoolatjoint(v2, v4)>,
 <Atom notoolatjoint(v4, v2)>,
 <Atom joint(v2, v5)>,
 <Atom joint(v5, v2)>,
 <Atom notoolatjoint(v2, v5)>,
 <Atom notoolatjoint(v5, v2)>,
 <Atom joint(

In [91]:
from pddlstream.language.conversion import value_from_evaluation

solution.certificate.all_facts

[('robotconf', Conf-Home),
 ('robotatconf', Conf-Home),
 ('canfreemove',),
 ('robottoolchangerempty',),
 ('element', 'b0'),
 ('atrack', 'b0'),
 ('iselement', 'b0'),
 ('grounded', 'b0'),
 ('element', 'b1'),
 ('atrack', 'b1'),
 ('iselement', 'b1'),
 ('grounded', 'b1'),
 ('element', 'b2'),
 ('atrack', 'b2'),
 ('iselement', 'b2'),
 ('element', 'b3'),
 ('atrack', 'b3'),
 ('iselement', 'b3'),
 ('element', 'b5'),
 ('atrack', 'b5'),
 ('iselement', 'b5'),
 ('grounded', 'b5'),
 ('element', 'b6'),
 ('atrack', 'b6'),
 ('iselement', 'b6'),
 ('element', 'b7'),
 ('atrack', 'b7'),
 ('iselement', 'b7'),
 ('grounded', 'b7'),
 ('element', 'b8'),
 ('atrack', 'b8'),
 ('iselement', 'b8'),
 ('element', 'b4'),
 ('atrack', 'b4'),
 ('iselement', 'b4'),
 ('joint', 'b0', 'b2'),
 ('joint', 'b2', 'b0'),
 ('notoolatjoint', 'b0', 'b2'),
 ('notoolatjoint', 'b2', 'b0'),
 ('joint', 'b0', 'b3'),
 ('joint', 'b3', 'b0'),
 ('notoolatjoint', 'b0', 'b3'),
 ('notoolatjoint', 'b3', 'b0'),
 ('joint', 'b0', 'b6'),
 ('joint', 'b6'

# pyplanner debuggings

In [490]:
debug_problem_name = "nine_pieces_process_symbolic.json"
input_problem, gripper_from_beam, clamp_from_joint = get_itj_pddl_problem_from_json(debug_problem_name, use_partial_order=True, debug=True, 
                                                    reset_to_home=0,
                                                    use_fluents=1)

Symbolic process json parsed from C:\Users\harry\Documents\code_ws\pb_ws\debug_pyplanners\itj\nine_pieces_process_symbolic.json
Using beam sequence ordering: ['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']


In [480]:
from pddlstream.language.attachments import has_attachments, compile_fluents_as_attachments, solve_pyplanners

evaluations, goal_expression, domain, externals = parse_problem(
    input_problem, constraints=constraints, unit_costs=unit_costs)
static_externals = compile_fluents_as_attachments(domain, externals)

In [481]:
from pddlstream.algorithms.downward import get_problem, task_from_domain_problem

# pddlstream.algorithns.downward.Problem
problem = get_problem(evaluations, goal_expression, domain, unit_costs)

# convert to pddl.Task
task = task_from_domain_problem(domain, problem)

In [482]:
domain.axioms[0].condition.parts[1]

<NegatedAtom new-axiom@2(?element)>

In [483]:
from pddlstream.algorithms.instantiate_task import sas_from_pddl, instantiate_task
from pddlstream.algorithms.instantiate_task import convert_instantiated

with Verbose(1):
    instantiated = instantiate_task(task)
    instantiated = convert_instantiated(instantiated)


Generating Datalog program... [0.016s CPU, 0.003s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.002s wall-clock]
Preparing model... [0.000s CPU, 0.005s wall-clock]
Generated 48 rules.
Computing model... [0.016s CPU, 0.010s wall-clock]
969 relevant atoms
35 auxiliary atoms
1004 final queue length
2313 total queue pushes
Completing instantiation... [0.016s CPU, 0.019s wall-clock]
Infeasible: False
Instantiation time: 0.045s
Instantiated frequencies:
Atoms: {alltoolatjoints: 17, assembled: 17, atrack: 17, attached: 17, canfreemove: 1, eitherassembled: 20, eithergroundedalltoolatjoints: 17, new-axiom@0: 8, new-axiom@1: 20, new-axiom@2: 9, notoolatjoint: 20, robotgripperempty: 1, robottoolchangerempty: 1, toolatjoint: 40, toolnotoccupiedonjoint: 4}
Actions: {pick_clamp_from_joint: 40, pick_element_from_rack: 9, pick_tool_from_rack: 7, place_clamp_at_joint: 40, place_element_on_structure: 27, place_tool_at_rack: 11}
Axioms: {alltoolatjoints: 9, eithe

In [495]:
clamp_from_joint['b1,b3']

{'c3', 'c4'}

In [485]:
instantiated.axioms

[<PropositionalAxiom (new-axiom@0 v9) [<NegatedAtom assembled(v8)>] -> Atom new-axiom@0(v9)>,
 <PropositionalAxiom (new-axiom@0 v9) [<Atom assembled(v8)>] -> NegatedAtom new-axiom@0(v9)>,
 <PropositionalAxiom (new-axiom@0 v8) [<NegatedAtom assembled(v7)>] -> Atom new-axiom@0(v8)>,
 <PropositionalAxiom (new-axiom@0 v8) [<Atom assembled(v7)>] -> NegatedAtom new-axiom@0(v8)>,
 <PropositionalAxiom (new-axiom@0 v7) [<NegatedAtom assembled(v6)>] -> Atom new-axiom@0(v7)>,
 <PropositionalAxiom (new-axiom@0 v7) [<Atom assembled(v6)>] -> NegatedAtom new-axiom@0(v7)>,
 <PropositionalAxiom (new-axiom@0 v6) [<NegatedAtom assembled(v5)>] -> Atom new-axiom@0(v6)>,
 <PropositionalAxiom (new-axiom@0 v6) [<Atom assembled(v5)>] -> NegatedAtom new-axiom@0(v6)>,
 <PropositionalAxiom (new-axiom@0 v5) [<NegatedAtom assembled(v4)>] -> Atom new-axiom@0(v5)>,
 <PropositionalAxiom (new-axiom@0 v5) [<Atom assembled(v4)>] -> NegatedAtom new-axiom@0(v5)>,
 <PropositionalAxiom (new-axiom@0 v4) [<NegatedAtom assemble

In [381]:
import instantiate
# FD instantiate
relaxed_reachable, atoms, actions, axioms, reachable_action_params = instantiate.explore(task)
# * add negated atoms of axiom precondition to atoms
negated_atoms = set()
for ax in axioms:
    negated_atoms.update({ng_atom.positive() for ng_atom in ax.condition if ng_atom.negated})
atoms.update(negated_atoms)

Generating Datalog program... [0.000s CPU, 0.001s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.001s wall-clock]
Preparing model... [0.000s CPU, 0.002s wall-clock]
Generated 48 rules.
Computing model... [0.016s CPU, 0.007s wall-clock]
969 relevant atoms
35 auxiliary atoms
1004 final queue length
2313 total queue pushes
Completing instantiation... [0.016s CPU, 0.014s wall-clock]


In [360]:
# unsatisfied[axiom] = len(axiom.conditions)
# for literal in axiom.conditions:
#     unprocessed[literal].append(axiom)

# reached_literals = {literal for literal in unprocessed if literal in state}
# queue = deque(reached_literals)

na = list(negated_atoms)[10]
# na = list(ax.condition)[0]
# na.negated
na.dump()

  NegatedAtom new-axiom@2(v3)


In [313]:
ax.condition

[<NegatedAtom new-axiom@2(v1)>]

In [358]:
at = list(atoms)[0]
at.negated

False

In [382]:
from pddlstream.language.attachments import DEFAULT_PYPLANNER, get_attachment_test
from strips.states import State, PartialState
from strips.operators import Action, Axiom

pyplanner = dict(DEFAULT_PYPLANNER)

fd_action_from_py_action = {}
py_actions = []
for action in instantiated.actions:
    # action.dump()
    py_action = Action({'fd_action': action})
    py_action.conditions = set(action.precondition)
    
    py_action.effects = set()
    for condition, effect in action.del_effects:
        assert not condition
        py_action.effects.add(effect.negate())
    for condition, effect in action.add_effects:
        assert not condition
        py_action.effects.add(effect)
    py_action.cost = action.cost
    py_action.test, fd_action_from_py_action[py_action] = get_attachment_test(action)
    py_actions.append(py_action)

In [396]:
instantiated.task.init

[<Atom robottoolchangerempty()>,
 <Atom element(v1)>,
 <Atom atrack(v1)>,
 <Atom iselement(v1)>,
 <Atom grounded(v1)>,
 <Atom element(v2)>,
 <Atom atrack(v2)>,
 <Atom iselement(v2)>,
 <Atom grounded(v2)>,
 <Atom element(v3)>,
 <Atom atrack(v3)>,
 <Atom iselement(v3)>,
 <Atom element(v4)>,
 <Atom atrack(v4)>,
 <Atom iselement(v4)>,
 <Atom element(v5)>,
 <Atom atrack(v5)>,
 <Atom iselement(v5)>,
 <Atom grounded(v5)>,
 <Atom element(v6)>,
 <Atom atrack(v6)>,
 <Atom iselement(v6)>,
 <Atom element(v7)>,
 <Atom atrack(v7)>,
 <Atom iselement(v7)>,
 <Atom grounded(v7)>,
 <Atom element(v8)>,
 <Atom atrack(v8)>,
 <Atom iselement(v8)>,
 <Atom element(v9)>,
 <Atom atrack(v9)>,
 <Atom iselement(v9)>,
 <Atom joint(v1, v3)>,
 <Atom joint(v3, v1)>,
 <Atom notoolatjoint(v1, v3)>,
 <Atom notoolatjoint(v3, v1)>,
 <Atom joint(v1, v4)>,
 <Atom joint(v4, v1)>,
 <Atom notoolatjoint(v1, v4)>,
 <Atom notoolatjoint(v4, v1)>,
 <Atom joint(v1, v6)>,
 <Atom joint(v6, v1)>,
 <Atom notoolatjoint(v1, v6)>,
 <Atom not

In [384]:
# ! both Axiom and Action are "Operator"s
# converting pddl axioms to strips.operator.Axioms
py_axioms = []
for axiom in instantiated.axioms:
    # axiom.dump()
    py_axiom = Axiom({'fd_axiom_id': id(axiom)}) # Not hashable for some reason
    py_axiom.conditions = set(axiom.condition)
    py_axiom.effects = {axiom.effect}
    py_axioms.append(py_axiom)

In [434]:
goal = PartialState(instantiated.goal_list)
fluents = {f.positive() for f in goal.conditions}
for py_operator in py_actions + py_axioms:
    fluents.update(f.positive() for f in py_operator.conditions)

# construct initial state

In [473]:
import pddl

intial_atoms = {atom for atom in instantiated.task.init \
        if isinstance(atom, pddl.Atom) and (atom in fluents)}
    
# * add negated atoms of axiom precondition to atoms
for ax in py_axioms:
    intial_atoms.update({ng_atom for ng_atom in ax.conditions \
                        if ng_atom.negated 
                         # and \
                         # (ng_atom in fluents)
                        # not (ng_atom.positive() in instantiated.goal_list) and \
                        })
    
initial = State(intial_atoms)

In [474]:
initial

State{Atom atrack(v1), Atom atrack(v10), Atom atrack(v11), Atom atrack(v12), Atom atrack(v13), Atom atrack(v14), Atom atrack(v15), Atom atrack(v16), Atom atrack(v2), Atom atrack(v3), Atom atrack(v4), Atom atrack(v5), Atom atrack(v6), Atom atrack(v7), Atom atrack(v8), Atom atrack(v9), Atom notoolatjoint(v1, v3), Atom notoolatjoint(v1, v4), Atom notoolatjoint(v1, v6), Atom notoolatjoint(v2, v3), Atom notoolatjoint(v2, v4), Atom notoolatjoint(v2, v8), Atom notoolatjoint(v3, v1), Atom notoolatjoint(v3, v2), Atom notoolatjoint(v4, v1), Atom notoolatjoint(v4, v2), Atom notoolatjoint(v5, v6), Atom notoolatjoint(v5, v9), Atom notoolatjoint(v6, v1), Atom notoolatjoint(v6, v5), Atom notoolatjoint(v7, v8), Atom notoolatjoint(v7, v9), Atom notoolatjoint(v8, v2), Atom notoolatjoint(v8, v7), Atom notoolatjoint(v9, v5), Atom notoolatjoint(v9, v7), Atom robottoolchangerempty(), Atom toolnotoccupiedonjoint(v10), Atom toolnotoccupiedonjoint(v11), Atom toolnotoccupiedonjoint(v12), Atom toolnotoccupiedonj

In [475]:
na = list(negated_atoms)[10] # 10
na.dump()

  NegatedAtom new-axiom@2(v3)


In [476]:
na.negated and (na.positive() not in initial)

True

In [442]:
na.positive() in instantiated.goal_list

True

In [477]:
derived_sv, ax_seq = derive_predicates(initial, py_axioms)

In [478]:
from utils import print_state
print_state(initial)
print_state(derived_sv)

State: 
assembled[b0]	assembled[b1]	assembled[b2]	assembled[b3]	assembled[b5]	assembled[b6]	
assembled[b7]	assembled[b8]	atrack[b0]	atrack[b1]	atrack[b2]	atrack[b3]	
atrack[b4]	atrack[b5]	atrack[b6]	atrack[b7]	atrack[b8]	atrack[c1]	
atrack[c2]	atrack[c3]	atrack[c4]	atrack[g1]	atrack[g2]	atrack[g3]	
new-axiom@1[b0, b2]	new-axiom@1[b0, b3]	new-axiom@1[b0, b6]	new-axiom@1[b1, b2]	new-axiom@1[b1, b3]	new-axiom@1[b1, b8]	
new-axiom@1[b2, b0]	new-axiom@1[b2, b1]	new-axiom@1[b3, b0]	new-axiom@1[b3, b1]	new-axiom@1[b4, b5]	new-axiom@1[b4, b7]	
new-axiom@1[b5, b4]	new-axiom@1[b5, b6]	new-axiom@1[b6, b0]	new-axiom@1[b6, b5]	new-axiom@1[b7, b4]	new-axiom@1[b7, b8]	
new-axiom@1[b8, b1]	new-axiom@1[b8, b7]	new-axiom@2[b0]	new-axiom@2[b1]	new-axiom@2[b2]	new-axiom@2[b3]	
new-axiom@2[b4]	new-axiom@2[b5]	new-axiom@2[b6]	new-axiom@2[b7]	new-axiom@2[b8]	notoolatjoint[b0, b2]	
notoolatjoint[b0, b3]	notoolatjoint[b0, b6]	notoolatjoint[b1, b2]	notoolatjoint[b1, b3]	notoolatjoint[b1, b8]	notoolatjoint[b2, b

In [387]:
from strips.utils import SEARCHES, EVALUATORS, SUCCESSORS, HEURISTICS, lookup_heuristic, pair_h_and_ha, single_generator

search='eager'
evaluator='greedy'
heuristic='ff'
successors='all'
    
search_fn = SEARCHES[search]
evaluator_fn = EVALUATORS[evaluator]

heuristic_fn = lookup_heuristic(HEURISTICS[heuristic])
successor_fn = SUCCESSORS[successors]
combined_fn = pair_h_and_ha(heuristic_fn, successor_fn)

generator_fn = single_generator(goal, py_actions, py_axioms, combined_fn)

In [388]:
from planner.state_space import test_goal, test_parent_operator, StateSpace

space = StateSpace(generator_fn, initial, max_extensions=INF)

In [389]:
sv = space.root
sv.generate()

Iterations: 0 | State Space: 1 | Expanded: 0 | Generations: 0 | Heuristic: 20 | Time: 0.397


True

In [391]:
from planner.operators import derive_predicates

solution = sv.state_space.solution(sv)
state_history = solution.plan.get_states()
plan = solution.plan

In [392]:
derived_sv, ax_seq = derive_predicates(sv.state, sv.state_space.axioms)

In [393]:
from utils import print_state
print_state(sv.state)
print_state(derived_sv)

State: 
atrack[b0]	atrack[b1]	atrack[b2]	atrack[b3]	atrack[b4]	atrack[b5]	
atrack[b6]	atrack[b7]	atrack[b8]	atrack[c1]	atrack[c2]	atrack[c3]	
atrack[c4]	atrack[g1]	atrack[g2]	atrack[g3]	notoolatjoint[b0, b2]	notoolatjoint[b0, b3]	
notoolatjoint[b0, b6]	notoolatjoint[b1, b2]	notoolatjoint[b1, b3]	notoolatjoint[b1, b8]	notoolatjoint[b2, b0]	notoolatjoint[b2, b1]	
notoolatjoint[b3, b0]	notoolatjoint[b3, b1]	notoolatjoint[b4, b5]	notoolatjoint[b4, b7]	notoolatjoint[b5, b4]	notoolatjoint[b5, b6]	
notoolatjoint[b6, b0]	notoolatjoint[b6, b5]	notoolatjoint[b7, b4]	notoolatjoint[b7, b8]	notoolatjoint[b8, b1]	notoolatjoint[b8, b7]	
robottoolchangerempty[]	toolnotoccupiedonjoint[c1]	toolnotoccupiedonjoint[c2]	toolnotoccupiedonjoint[c3]	toolnotoccupiedonjoint[c4]	
State: 
alltoolatjoints[b0]	alltoolatjoints[b1]	alltoolatjoints[b2]	alltoolatjoints[b3]	alltoolatjoints[b4]	alltoolatjoints[b5]	
alltoolatjoints[b6]	alltoolatjoints[b7]	alltoolatjoints[b8]	atrack[b0]	atrack[b1]	atrack[b2]	
atrack[b3]	atr

In [374]:
from collections import defaultdict, deque

state, axioms = sv.state, axioms

unprocessed = defaultdict(list)
unsatisfied = {}
for axiom in axioms:
    unsatisfied[axiom] = len(axiom.conditions)
    for literal in axiom.conditions:
        unprocessed[literal].append(axiom)

reached_literals = {literal for literal in unprocessed if literal in state}
queue = deque(reached_literals)

In [375]:
na in state

True

In [376]:
na.positive()

<Atom new-axiom@2(v3)>

In [377]:
na.negated

True

In [378]:
na.positive() not in state

True

In [279]:
list(goal)[0].negated

False

In [344]:
list(state.atoms)[0].positive()

<Atom atrack(v1)>

In [345]:
list(reached_literals)

[<NegatedAtom assembled(v1)>,
 <NegatedAtom new-axiom@1(v4, v1)>,
 <NegatedAtom new-axiom@1(v8, v7)>,
 <NegatedAtom new-axiom@1(v1, v6)>,
 <NegatedAtom new-axiom@2(v3)>,
 <NegatedAtom new-axiom@1(v2, v4)>,
 <NegatedAtom assembled(v3)>,
 <NegatedAtom new-axiom@1(v4, v2)>,
 <NegatedAtom new-axiom@1(v9, v7)>,
 <NegatedAtom assembled(v5)>,
 <NegatedAtom assembled(v2)>,
 <NegatedAtom new-axiom@1(v2, v8)>,
 <NegatedAtom new-axiom@2(v4)>,
 <NegatedAtom new-axiom@1(v5, v9)>,
 <NegatedAtom new-axiom@1(v1, v4)>,
 <NegatedAtom new-axiom@1(v7, v8)>,
 <NegatedAtom new-axiom@1(v1, v3)>,
 <NegatedAtom new-axiom@1(v3, v2)>,
 <NegatedAtom new-axiom@1(v6, v1)>,
 <NegatedAtom assembled(v7)>,
 <NegatedAtom assembled(v6)>,
 <NegatedAtom new-axiom@2(v5)>,
 <NegatedAtom new-axiom@2(v9)>,
 <NegatedAtom new-axiom@1(v6, v5)>,
 <NegatedAtom new-axiom@1(v9, v5)>,
 <NegatedAtom new-axiom@2(v1)>,
 <NegatedAtom new-axiom@1(v3, v1)>,
 <NegatedAtom new-axiom@2(v6)>,
 <NegatedAtom new-axiom@2(v7)>,
 <NegatedAtom new-ax

In [196]:
len(state_history)

1

In [203]:
# for ax in py_axioms:
#     ax.dump()

In [346]:
list(derived_sv.atoms)[25].dump()

  Atom atrack(v2)


In [349]:
print_state(sv.state)
print_state(sv.derived_state)

State: 
atrack[b0]	atrack[b1]	atrack[b2]	atrack[b3]	atrack[b4]	atrack[b5]	
atrack[b6]	atrack[b7]	atrack[b8]	atrack[c1]	atrack[c2]	atrack[c3]	
atrack[c4]	atrack[g1]	atrack[g2]	atrack[g3]	notoolatjoint[b0, b2]	notoolatjoint[b0, b3]	
notoolatjoint[b0, b6]	notoolatjoint[b1, b2]	notoolatjoint[b1, b3]	notoolatjoint[b1, b8]	notoolatjoint[b2, b0]	notoolatjoint[b2, b1]	
notoolatjoint[b3, b0]	notoolatjoint[b3, b1]	notoolatjoint[b4, b5]	notoolatjoint[b4, b7]	notoolatjoint[b5, b4]	notoolatjoint[b5, b6]	
notoolatjoint[b6, b0]	notoolatjoint[b6, b5]	notoolatjoint[b7, b4]	notoolatjoint[b7, b8]	notoolatjoint[b8, b1]	notoolatjoint[b8, b7]	
robottoolchangerempty[]	toolnotoccupiedonjoint[c1]	toolnotoccupiedonjoint[c2]	toolnotoccupiedonjoint[c3]	toolnotoccupiedonjoint[c4]	
State: 
alltoolatjoints[b0]	alltoolatjoints[b1]	alltoolatjoints[b2]	alltoolatjoints[b3]	alltoolatjoints[b4]	alltoolatjoints[b5]	
alltoolatjoints[b6]	alltoolatjoints[b7]	alltoolatjoints[b8]	atrack[b0]	atrack[b1]	atrack[b2]	
atrack[b3]	atr

In [200]:
from strips.utils import solve_strips
plan, state_space = solve_strips(initial, goal, py_actions, py_axioms)

strips search
Search: eager | Evaluator: greedy | Heuristic: ff | Successors: all

-------------------------------------------------------------------------------------

State: Statespace Root
atrack[b0]	atrack[b1]	atrack[b2]	atrack[b3]	atrack[b4]	atrack[b5]	
atrack[b6]	atrack[b7]	atrack[b8]	atrack[c1]	atrack[c2]	atrack[c3]	
atrack[c4]	atrack[g1]	atrack[g2]	atrack[g3]	notoolatjoint[b0, b2]	notoolatjoint[b0, b3]	
notoolatjoint[b0, b6]	notoolatjoint[b1, b2]	notoolatjoint[b1, b3]	notoolatjoint[b1, b8]	notoolatjoint[b2, b0]	notoolatjoint[b2, b1]	
notoolatjoint[b3, b0]	notoolatjoint[b3, b1]	notoolatjoint[b4, b5]	notoolatjoint[b4, b7]	notoolatjoint[b5, b4]	notoolatjoint[b5, b6]	
notoolatjoint[b6, b0]	notoolatjoint[b6, b5]	notoolatjoint[b7, b4]	notoolatjoint[b7, b8]	notoolatjoint[b8, b1]	notoolatjoint[b8, b7]	
robottoolchangerempty[]	toolnotoccupiedonjoint[c1]	toolnotoccupiedonjoint[c2]	toolnotoccupiedonjoint[c3]	toolnotoccupiedonjoint[c4]	
+++++
pick_tool_from_rack[c1]
[[atrack, c1], [robott

In [78]:
# py_actions
py_axioms[0].dump()

Axiom{fd_axiom_id: 2083305731144}
pre: set()
eff: {<Atom connected(v1)>}
cost: 0


In [103]:
from strips.ff import ff_fn, plan_cost, first_goals
from strips.utils import single_generator

combined_fn = ff_fn(plan_cost, first_goals, op=sum)
generator_fn = single_generator(goal, py_actions, py_axioms, combined_fn)

In [111]:
from misc.objects import str_object

# print(str_object(py_axioms[1].args))
print(py_axioms[1].args['fd_axiom_id'])

2083307685448


In [ ]:
from pddlstream.algorithms.incremental import solve_incremental
from pddlstream.language.constants import print_plan, is_plan

solution = solve_incremental(input_problem, constraints=constraints,
                  unit_costs=1, success_cost=INF,
                  max_iterations=INF, max_time=INF, max_memory=INF,
                  initial_complexity=0, complexity_step=1, max_complexity=INF,
                  verbose=0)

plan, cost, evaluations = solution
plan_success = is_plan(plan)

print('-'*10)
print_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

Iteration: 1 | Complexity: 0 | Calls: 0 | Evaluations: 161 | Solved: False | Cost: inf | Search Time: 0.003 | Sample Time: 0.000 | Time: 0.003
Initial: State{Atom notoolatjoint(v1, v3), Atom notoolatjoint(v1, v4), Atom notoolatjoint(v1, v6), Atom notoolatjoint(v2, v3), Atom notoolatjoint(v2, v4), Atom notoolatjoint(v2, v8), Atom notoolatjoint(v3, v1), Atom notoolatjoint(v3, v2), Atom notoolatjoint(v4, v1), Atom notoolatjoint(v4, v2), Atom notoolatjoint(v5, v6), Atom notoolatjoint(v5, v9), Atom notoolatjoint(v6, v1), Atom notoolatjoint(v6, v5), Atom notoolatjoint(v7, v8), Atom notoolatjoint(v7, v9), Atom notoolatjoint(v8, v2), Atom notoolatjoint(v8, v7), Atom notoolatjoint(v9, v5), Atom notoolatjoint(v9, v7)}
Goal: PartialState{Atom assembled(v1), Atom assembled(v2), Atom assembled(v3), Atom assembled(v4), Atom assembled(v5), Atom assembled(v6), Atom assembled(v7), Atom assembled(v8), Atom assembled(v9)}
Actions: 0 | Axioms: 73
Search: eager | Evaluator: greedy | Heuristic: ff | Success